# [week4 _ assignment]
### [휴먼지능정보공학과 201810756 김대환]
---

##### [문제 : 1번]

2건의 데이터에 대해 Me"rkle Root 값 계산하고 출력
* txA='Hello'
* txB='How are you?'

In [5]:
import hashlib
txA='Hello'
txB='How are you?'
# 각각의 문자열을 재해싱 -> _hA,_hB / 왜 더블해싱 ? : 'length extension attack' 방지 
_hA=hashlib.sha256(hashlib.sha256(txA.encode()).digest()).hexdigest()
_hB=hashlib.sha256(hashlib.sha256(txB.encode()).digest()).hexdigest()
print("_hA : ",_hA)
print("_hB : ",_hB)
print("----------------------------------------------")
# 바이트 교환 
hA="".join(reversed([_hA[i:i+2]for i in range(0,32*2,2)]))
hB="".join(reversed([_hB[i:i+2]for i in range(0,32*2,2)]))
print("hA : ",hA)
print("hB : ",hB)
print("----------------------------------------------")
# hA + hB
_hAhB=hA+hB
print("_hAhB : ",_hAhB)
print("----------------------------------------------")
# _hAhB 재해싱(더블)
_hAB=hashlib.sha256(hashlib.sha256(_hAhB.encode()).digest()).hexdigest()
print("_hAB : ",_hAB)
print("----------------------------------------------")
# 바이트 교환
hAB="".join(reversed([_hAB[i:i+2] for i in range(0,32*2,2)]))
print("최종 merkle root : ", hAB)

_hA :  70bc18bef5ae66b72d1995f8db90a583a60d77b4066e4653f1cead613025861c
_hB :  1027ce357faf49fc632d35908db181ae951aa3df9060595d94fb33769d71e7d0
----------------------------------------------
hA :  1c86253061adcef153466e06b4770da683a590dbf895192db766aef5be18bc70
hB :  d0e7719d7633fb945d596090dfa31a95ae81b18d90352d63fc49af7f35ce2710
----------------------------------------------
_hAhB :  1c86253061adcef153466e06b4770da683a590dbf895192db766aef5be18bc70d0e7719d7633fb945d596090dfa31a95ae81b18d90352d63fc49af7f35ce2710
----------------------------------------------
_hAB :  e0c76d87a5a5c18ab29757603c5d1bda709306203b0a44c53fc6c90fba162903
----------------------------------------------
최종 merkle root :  032916ba0fc9c63fc5440a3b20069370da1b5d3c605797b28ac1a5a5876dc7e0


##### [문제 : 2번]

친구에게 주소를 구하고, 그 주소로 송금\
송금이 되지 않는다면 왜 안되는지 이유를 알아보자.\
안되면 친구의 주소를 만들고 전송한다.
* 잔고의 증가분
* 소요된 gas비용

In [6]:
%%writefile src/w4_assignment_testTransaction.js
var hqaccount="0x4fa2c7caac80a8518264d263bdb5ed74f1a6f398"; // 상대방 주소 
console.log('거래 전 상대방 잔고 : ',eth.getBalance(hqaccount));
var before_balance=eth.getBalance(hqaccount);
console.log('block number : ',eth.blockNumber);
var t=eth.sendTransaction({from:eth.accounts[0],to:hqaccount,value:500});
console.log('transactionHash : ',t);
console.log('..mining start');
miner.start(1);admin.sleepBlocks(1);miner.stop();
console.log('mining done..');
console.log('block number : ',eth.blockNumber);
var after_balance=eth.getBalance(hqaccount);
console.log('거래 후 상대방 잔고 : ',eth.getBalance(hqaccount));
console.log ('잔고 증가분 : ',after_balance-before_balance);
console.log('소요된 gas 비용 : ',eth.getTransactionReceipt(t).gasUsed);

Overwriting src/w4_assignment_testTransaction.js


In [8]:
# w4_assignment_testTransaction.js 실행
!geth --exec 'loadScript("src/w4_assignment_testTransaction.js")' attach http://localhost:8445

거래 전 상대방 잔고 :  6000
block number :  3140
transactionHash :  0x03a54f2fc348dbd062a795cee17b789c89f4867b623e98e5ea39f4a9d1d8f6a3
..mining start
mining done..
block number :  3141
거래 후 상대방 잔고 :  6500
잔고 증가분 :  500
소요된 gas 비용 :  21000
null


상대방에게 송금이 잘 되었음을 확인\
목표한 금액(500) 만큼 상대방의 잔고가 늘었음

##### [문제 : 3번]

블록헤더 데이터의 해시 값에 NONCE를 증가시키면서
앞 자리의 0의 개수를 맞출 때까지 반복한다.
1. 찾고자하는 해시 '''0000'''로 시작, 몇 회만에 찾는지 출력
2. 찾고자하는 해시 '''00000'''로 시작, 몇 회만에 찾는지 출력
3. 찾고자하는 해시 '''000000'''로 시작, 몇 회만에 찾는지 출력

In [9]:
# 적체되어 있는 거래 확인
!geth --exec txpool.inspect attach http://localhost:8445

{
  pending: {},
  queued: {}
}


In [10]:
# 해시 맞추기 target : "0000" / 시작 해시 : 1번 문제에서 구한 merkle root로 임의설정
ntry=1
found=False
blockNumber=54 
NONCE=0
data='hello'
previousHash='032916ba0fc9c63fc5440a3b20069370da1b5d3c605797b28ac1a5a5876dc7e0'
while found==False:
    z=str(blockNumber)+str(NONCE)+data+previousHash
    guessHash=hashlib.sha256(z.encode()).hexdigest()
    if guessHash[:4]=='0000':
        found=True
    NONCE+=1
    if(NONCE%10000000==0):
        print("NONCE: ",NONCE, guessHash)
print("solved ","Nonce : ",NONCE,"guessHash : ",guessHash)

solved  Nonce :  23021 guessHash :  00004b3bc24b04b605412a973e2b8c754fb8087f00432c6a029bd67d79605bf8


In [11]:
# target : "00000"
ntry=1
found=False
blockNumber=54 
NONCE=0
data='hello'
previousHash='032916ba0fc9c63fc5440a3b20069370da1b5d3c605797b28ac1a5a5876dc7e0'
while found==False:
    z=str(blockNumber)+str(NONCE)+data+previousHash
    guessHash=hashlib.sha256(z.encode()).hexdigest()
    if guessHash[:5]=='00000':
        found=True
    NONCE+=1
    if(NONCE%10000000==0):
        print("NONCE: ",NONCE, guessHash)
print("solved ","Nonce : ",NONCE,"guessHash : ",guessHash)

solved  Nonce :  966124 guessHash :  00000e9f000080833cd40b0c3cc3fea979afefdad55bfd2057e1335a279f5709


In [12]:
# target : "000000"
ntry=1
found=False
blockNumber=54 
NONCE=0
data='hello'
previousHash='032916ba0fc9c63fc5440a3b20069370da1b5d3c605797b28ac1a5a5876dc7e0'
while found==False:
    z=str(blockNumber)+str(NONCE)+data+previousHash
    guessHash=hashlib.sha256(z.encode()).hexdigest()
    if guessHash[:6]=='000000':
        found=True
    NONCE+=1
    if(NONCE%10000000==0):
        print("NONCE: ",NONCE, guessHash)
print("solved ","Nonce : ",NONCE,"guessHash : ",guessHash)

NONCE:  10000000 74a3f8f2af60a8ca2837f5c84edfe681683066da5be2a55e3858b7c47baf81e8
NONCE:  20000000 22832c44e0932d51f55b35c8c8302c32d34f0aefafa21af56701d2231d8dcb5d
NONCE:  30000000 052e776db155d4b8f98a9c1b27d7ec7aa4c7bdc1b745337ee68a28c7d88702ba
solved  Nonce :  38750291 guessHash :  000000eb5835517c61592ed42091047668ab738cc795d60ec36b26d8726d22bc


블록헤더의 Hash 값(guessHash) 이 찾고자하는 목표 해시값의 최대값보다 작은 범위안에 들게 되면 멈추는데\
목표 해시값이 "0000"인 경우 최대값은 "0000FFFF..." 이므로 이보다 작은 해시값을 가지는 NONCE를 출력(23021).\
"00000"인 경우 최대값은 "00000FFF..." , 이보다 작은 해시값을 가지는 NONCE 출력(966124)\
"000000"인 경우 최대값은 "000000FF...", 이보다 작은 해기값을 가지는 NONCE 출력(39750291)\
**NONCE가 곧 횟수**\
목표해시값이 작을수록 시간이 오래 걸리고 찾는 회수가 많이 소모된다.